In [ ]:
import torch
import pandas as pd
import numpy as np
from torch import nn
import os
import sklearn
import matplotlib.pyplot as plt
import sys
import re

In [ ]:
cwd = os.getcwd() + "/ACC"

print(cwd)

os.chdir(cwd)

## A Dataset with Accelerometer, Magnetometer, Gyroscope, Environment, and GPS Data [14]
### Each file comprises the data in a matrix format, with one line per sample, and one column per channel. The channels are as follows:
* Time of sample in millisecond
* Ankle (shank) acceleration - horizontal forward acceleration [mg]
* Ankle (shank) acceleration - vertical [mg]
* Ankle (shank) acceleration - horizontal lateral [mg]
* Upper leg (thigh) acceleration - horizontal forward acceleration [mg]
* Upper leg (thigh) acceleration - vertical [mg]
* Upper leg (thigh) acceleration - horizontal lateral [mg]
* Trunk acceleration - horizontal forward acceleration [mg]
* Trunk acceleration - vertical [mg]
* Trunk acceleration - horizontal lateral [mg]
* Annotation [0, 1, or 2]



### The meaning of the annotations are as follows:
* 0: not part of the experiment. For instance the sensors are installed on the user or the user is performing activities unrelated to the experimental protocol, such as debriefing
* 1: experiment, no freeze (can be any of stand, walk, turn)
* 2: freeze


In [ ]:
CONVERTED = False

if(not CONVERTED):
    path = "/home/jm/vscode/Python/pytorch/Fall Dectection/DaphnetFreezingGait/dataset"

    os.chdir(path)

    #gets data from file specified by file_path
    def get_contents(file_path):
        
        #chest sensor
        chest_x_accel = []
        chest_y_accel = []
        chest_z_accel = []
        
        #electrocardiogram signals
        ecs1 = []
        ecs2 = []
        
        #lower left ankle acceleration data
        l_ankle_x_accel = []
        l_ankle_y_accel = []
        l_ankle_z_accel = []
        
        #left ankle gyro data
        l_ankle_x_g = []
        l_ankle_y_g= []
        l_ankle_z_g = []
        
        #magnetometer from left-ankle
        l_ankle_x_mag = []
        l_ankle_y_mag = []
        l_ankle_z_mag = []
        
        #right lower arm accerleration data
        r_arm_x_accel = []
        r_arm_y_accel = []
        r_arm_z_accel = []

        #right lower arm gyro data
        r_arm_x_g = []
        r_arm_y_g = []
        r_arm_z_g = []
        
        #right lower arm magnetometer
        r_arm_x_mag = []
        r_arm_y_mag = []
        r_arm_z_mag = []
        
        #label
        actvities = []

        with open(file_path,'r') as file:

            # lines = file.readlines()[:]

            for line in file:
                data = line.split()
                
                #chest sensor
                chest_x_accel = np.append(chest_x_accel, np.float32(data[0]))
                chest_y_accel = np.append(chest_y_accel, np.float32(data[1]))
                chest_z_accel = np.append(chest_z_accel, np.float32(data[2]))

                #electrocardiogram signals
                ecs1 = np.append(ecs1, np.float32(data[3]))
                ecs2 = np.append(ecs2, np.float32(data[4]))
                
                #lower left ankle acceleration data
                l_ankle_x_accel = np.append(l_ankle_x_accel, np.float32(data[5]))
                l_ankle_y_accel = np.append(l_ankle_y_accel, np.float32(data[6]))
                l_ankle_z_accel = np.append(l_ankle_z_accel, np.float32(data[7]))
                
                #left ankle gyro data
                l_ankle_x_g = np.append(l_ankle_x_g, np.float32(data[8]))
                l_ankle_y_g = np.append(l_ankle_y_g, np.float32(data[9]))
                l_ankle_z_g = np.append(l_ankle_z_g, np.float32(data[10]))
                
                #magnetometer from left-ankle
                l_ankle_x_mag = np.append(l_ankle_x_mag, np.float32(data[11]))
                l_ankle_y_mag = np.append(l_ankle_y_mag, np.float32(data[12]))
                l_ankle_z_mag = np.append(l_ankle_z_mag, np.float32(data[13]))
                
                #right lower arm acceleration data
                r_arm_x_accel = np.append(r_arm_x_accel, np.float32(data[14]))
                r_arm_y_accel = np.append(r_arm_y_accel, np.float32(data[15]))
                r_arm_z_accel = np.append(r_arm_z_accel, np.float32(data[16]))
                
                #right lower arm gyro
                r_arm_x_g = np.append(r_arm_x_g, np.float32(data[17]))
                r_arm_y_g = np.append(r_arm_y_g, np.float32(data[18]))
                r_arm_z_g = np.append(r_arm_z_g, np.float32(data[19]))
                
                #right  lower arm magnetometer
                r_arm_x_mag = np.append(r_arm_x_mag, np.float32(data[20]))
                r_arm_y_mag = np.append(r_arm_y_mag, np.float32(data[21]))
                r_arm_z_mag = np.append(r_arm_z_mag, np.float32(data[22]))
                
                #label
                actvities = np.append(actvities, np.int32(data[23]))
                
                
        df = pd.DataFrame({
            'chest acceleration X-axis': chest_x_accel,
            'chest acceleration Y-axis': chest_y_accel,
            'chest acceleration Z-axis': chest_z_accel,
            'electrocardiogram signal 1': ecs1,
            'electrocardiogram signal 2': ecs2,
            'lower left ankle acceleration X-axis': l_ankle_x_accel,
            'lower left ankle acceleration Y-axis': l_ankle_y_accel,
            'lower left ankle acceleration Z-axis': l_ankle_z_accel,
            'lower left ankle gyro X-axis': l_ankle_x_g, 
            'lower left ankle gyro Y-axis': l_ankle_y_g,
            'lower left ankle gyro Z-axis': l_ankle_z_g,
            'lower left ankle magnetometer X-axis': l_ankle_x_mag,
            'lower left ankle magnetometer Y-axis': l_ankle_y_mag,
            'lower left ankle magnetometer Z-axis': l_ankle_y_mag,
            'lower right arm acceleration X-axis': r_arm_x_accel,
            'lower right arm acceleration Y-axis': r_arm_y_accel,
            'lower right arm acceleration Z-axis': r_arm_z_accel,
            'lower right arm gryo X-axis': r_arm_x_g,
            'lower right arm gryo Y-axis': r_arm_y_g,
            'lower right arm gryo Z-axis': r_arm_z_g,
            'lower right arm magnetometer X-axis': r_arm_x_mag,
            'lower right arm magnetometer Y-axis': r_arm_y_mag,
            'lower right arm magnetometer Z-axis': r_arm_z_mag,
            'label': actvities,
        })
                
        return df

## DaphNet Freezing of Gait [14]
